In [1]:
import numpy as np
import pandas as pd
import sklearn.datasets as dt
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from LVQ.lvq import LVQ3
from sklearn.preprocessing import OneHotEncoder

In [2]:
X, y = pd.read_csv("../data/raw/train_values.csv"), pd.read_csv("../data/raw/train_labels.csv")

In [3]:
X

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,1020144,10,804,1371,2,5,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,199685,25,211,5071,2,25,9,4,n,r,...,0,0,0,0,0,0,0,0,0,0
2,1038433,22,816,12104,2,10,5,4,t,r,...,0,0,0,0,0,0,0,0,0,0
3,461920,6,449,3741,1,10,8,3,t,r,...,0,0,0,0,0,0,0,0,0,0
4,957027,20,254,5266,2,10,10,6,n,w,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208475,788768,22,165,10190,2,30,5,4,t,r,...,0,0,0,0,0,0,0,0,0,0
208476,490379,27,269,5197,3,80,5,6,t,r,...,0,0,0,0,0,0,0,0,0,0
208477,294228,26,1259,10829,1,10,4,6,t,r,...,0,0,0,0,0,0,0,0,0,0
208478,464117,7,258,1869,1,15,6,3,t,r,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df = pd.merge(X, y,left_on="building_id", right_on="building_id").iloc[:,1:]

In [5]:
# df.to_csv("../data/raw/train_data.csv")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208480 entries, 0 to 208479
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   geo_level_1_id                          208480 non-null  int64 
 1   geo_level_2_id                          208480 non-null  int64 
 2   geo_level_3_id                          208480 non-null  int64 
 3   count_floors_pre_eq                     208480 non-null  int64 
 4   age                                     208480 non-null  int64 
 5   area_percentage                         208480 non-null  int64 
 6   height_percentage                       208480 non-null  int64 
 7   land_surface_condition                  208480 non-null  object
 8   foundation_type                         208480 non-null  object
 9   roof_type                               208480 non-null  object
 10  ground_floor_type                       208480 non-null 

In [7]:
df.iloc[:,-1]

0         2
1         2
2         2
3         2
4         3
         ..
208475    1
208476    2
208477    3
208478    3
208479    2
Name: damage_grade, Length: 208480, dtype: int64

In [8]:
X, y = df.iloc[:,:-1], df.iloc[:,-1]

In [9]:
X.iloc[:,:7] = X.iloc[:,:7].astype("float")

/tmp/ipykernel_43833/2387548514.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.iloc[:,:7] = X.iloc[:,:7].astype("float")


In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208480 entries, 0 to 208479
Data columns (total 38 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   geo_level_1_id                          208480 non-null  float64
 1   geo_level_2_id                          208480 non-null  float64
 2   geo_level_3_id                          208480 non-null  float64
 3   count_floors_pre_eq                     208480 non-null  float64
 4   age                                     208480 non-null  float64
 5   area_percentage                         208480 non-null  float64
 6   height_percentage                       208480 non-null  float64
 7   land_surface_condition                  208480 non-null  object 
 8   foundation_type                         208480 non-null  object 
 9   roof_type                               208480 non-null  object 
 10  ground_floor_type                       2084

In [11]:
X

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,10.0,804.0,1371.0,2.0,5.0,7.0,6.0,t,r,n,...,0,0,0,0,0,0,0,0,0,0
1,25.0,211.0,5071.0,2.0,25.0,9.0,4.0,n,r,q,...,0,0,0,0,0,0,0,0,0,0
2,22.0,816.0,12104.0,2.0,10.0,5.0,4.0,t,r,q,...,0,0,0,0,0,0,0,0,0,0
3,6.0,449.0,3741.0,1.0,10.0,8.0,3.0,t,r,q,...,0,0,0,0,0,0,0,0,0,0
4,20.0,254.0,5266.0,2.0,10.0,10.0,6.0,n,w,q,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208475,22.0,165.0,10190.0,2.0,30.0,5.0,4.0,t,r,n,...,0,0,0,0,0,0,0,0,0,0
208476,27.0,269.0,5197.0,3.0,80.0,5.0,6.0,t,r,q,...,0,0,0,0,0,0,0,0,0,0
208477,26.0,1259.0,10829.0,1.0,10.0,4.0,6.0,t,r,n,...,0,0,0,0,0,0,0,0,0,0
208478,7.0,258.0,1869.0,1.0,15.0,6.0,3.0,t,r,q,...,0,0,0,0,0,0,0,0,0,0


In [12]:
from prince import FAMD, MCA
famd = FAMD(n_components =24, n_iter = 100, copy=True, random_state = 100)

X_t = famd.fit_transform(X)

In [13]:
X_t

component,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-1.701051,-3.000050,-2.044228,5.179376,-1.641714,-0.457770,0.195021,-1.759325,5.259082,3.656520,...,1.297791,1.293592,-0.478879,1.813038,2.351215,-2.265450,1.769207,1.348172,-0.610724,0.317428
1,-4.309564,-2.507675,-2.193442,-2.617493,-1.636667,-0.327119,2.021620,0.281312,0.169960,0.107069,...,-0.872599,0.624577,0.927984,-0.852549,-0.806622,1.231107,2.419647,-3.181501,-1.394613,0.333703
2,-3.977083,-2.600835,-2.686598,-1.733000,-1.327434,-2.821173,1.717161,3.515147,1.285762,1.306356,...,0.513944,0.474292,-1.136969,-0.153152,-0.686537,0.015549,0.072799,-0.171169,-0.729217,0.031830
3,-1.720871,0.469089,-3.641277,-3.406394,3.505867,1.178325,-6.110034,0.078589,1.169676,-2.801258,...,-6.949557,-3.231875,0.795902,-2.524868,0.596994,4.971336,1.755325,-5.073979,-7.057358,1.017891
4,-3.523487,6.577175,22.226918,-7.553189,-4.212696,-8.171183,2.029656,-0.900401,-4.282372,-1.095052,...,-0.656671,2.878975,2.843507,0.524680,-1.647712,1.736363,3.264446,-2.874477,-1.384443,-0.373464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208475,-4.461848,0.238742,-0.735954,-2.028388,-1.794362,-0.910754,0.061090,-1.373690,-0.909750,0.533776,...,1.050659,1.470510,0.022835,0.621411,0.606955,-0.153931,-0.445806,1.312991,0.674023,-0.228695
208476,-2.381251,-4.217525,-1.092262,-0.399383,-1.553406,-2.127632,0.886747,5.193605,2.025575,3.147457,...,-7.030700,-2.139624,1.498278,-1.407787,-1.537363,-1.549798,0.516195,-0.573612,-0.775153,0.384502
208477,-3.066257,2.537775,-1.134772,-3.825351,1.743642,-0.105119,-4.270846,-2.584461,-1.082671,-0.569215,...,1.133111,0.894609,0.160814,1.189596,0.768193,-0.593882,-0.751255,1.575147,0.741468,-0.178820
208478,-2.572597,-0.301124,-3.120865,-3.591399,2.290917,-1.981141,-2.663281,2.321766,1.073716,0.206718,...,0.633954,-0.122328,-1.006651,0.451298,-0.545475,-0.459392,-0.229523,0.099146,-0.697393,0.091694


In [14]:
from imblearn.over_sampling import SMOTENC, ADASYN
from imblearn.under_sampling import TomekLinks, NeighbourhoodCleaningRule, CondensedNearestNeighbour
from imblearn.combine import SMOTETomek, SMOTEENN
# sm = ADASYN(random_state=20)
# um = TomekLinks()
# um = NeighbourhoodCleaningRule()
# um = CondensedNearestNeighbour(random_state=20)
cm = SMOTETomek(random_state=20)
# cm = SMOTEENN(random_state=20)

In [15]:
X_res, y_res = cm.fit_resample(X_t, y)

In [16]:
X_res

component,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-4.309564,-2.507675,-2.193442,-2.617493,-1.636667,-0.327119,2.021620,0.281312,0.169960,0.107069,...,-0.872599,0.624577,0.927984,-0.852549,-0.806622,1.231107,2.419647,-3.181501,-1.394613,0.333703
1,-3.977083,-2.600835,-2.686598,-1.733000,-1.327434,-2.821173,1.717161,3.515147,1.285762,1.306356,...,0.513944,0.474292,-1.136969,-0.153152,-0.686537,0.015549,0.072799,-0.171169,-0.729217,0.031830
2,-1.720871,0.469089,-3.641277,-3.406394,3.505867,1.178325,-6.110034,0.078589,1.169676,-2.801258,...,-6.949557,-3.231875,0.795902,-2.524868,0.596994,4.971336,1.755325,-5.073979,-7.057358,1.017891
3,-3.992385,-3.115595,-1.830131,-2.374022,-0.543896,0.627794,0.231131,-4.448583,0.952306,-1.579524,...,3.346737,-8.424009,-4.104996,0.555465,-1.285616,-2.066577,-5.938534,4.775898,0.200813,1.691504
4,1.898713,5.260786,-2.488995,22.690547,-4.781668,-2.984109,4.551011,6.163579,-6.857970,-17.571726,...,7.746705,-9.384844,13.015751,-1.471932,5.756446,0.453969,-3.663084,-8.120956,7.137709,-3.938807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329411,-3.401530,-1.495000,-2.030445,-1.070015,-2.337376,0.105928,6.172387,9.258242,2.901840,0.436430,...,-0.576452,-2.686028,-1.357396,-4.320853,-1.034170,4.118631,-1.126499,0.083985,-1.150828,0.733957
329412,-3.352802,-2.784479,-2.287636,-1.680437,1.820542,2.200988,4.574798,-5.136315,0.601303,-2.785517,...,5.737044,-10.150256,-4.248440,0.034531,-1.433955,-4.791628,-5.851660,5.222713,-1.644472,1.694003
329413,-2.383247,-8.994809,13.442571,-0.299157,2.803459,-5.000589,0.271968,4.587223,-1.305468,-0.435320,...,1.345422,2.018589,-0.651736,1.010935,-1.785942,1.274840,0.326993,-0.428244,-1.137240,-0.265516
329414,-3.892260,-3.426427,-2.848297,-0.798657,-0.081187,-0.884771,-1.251743,-0.385185,0.063031,0.426799,...,0.926782,0.884333,-0.484270,0.498629,0.282707,0.018201,-0.351974,0.816108,0.532853,-0.150579


In [17]:
y_res

0         2
1         2
2         2
3         2
4         3
         ..
329411    3
329412    3
329413    3
329414    3
329415    3
Name: damage_grade, Length: 329416, dtype: int64

In [18]:
np.unique(y_res, return_counts=True)

(array([1, 2, 3]), array([115606, 106019, 107791]))

In [19]:
X_scaled = StandardScaler().fit_transform(X_res.values)
X_scaled

array([[-0.60331169, -0.47878823, -0.37686367, ..., -0.77680124,
        -0.31289417,  0.07488915],
       [-0.57485455, -0.49327078, -0.46532688, ..., -0.04818048,
        -0.16577757,  0.00801116],
       [-0.38174487, -0.01602149, -0.63657898, ..., -1.23485677,
        -1.56490598,  0.22646677],
       ...,
       [-0.43843775, -1.48727594,  2.42795336, ..., -0.11040282,
        -0.25599001, -0.05786394],
       [-0.56759454, -0.62161712, -0.4943327 , ...,  0.19078009,
         0.11326135, -0.03240049],
       [-0.02274558,  2.63868862,  1.4403692 , ...,  0.29441444,
         0.77594252,  0.07100784]])

In [20]:
y_res

0         2
1         2
2         2
3         2
4         3
         ..
329411    3
329412    3
329413    3
329414    3
329415    3
Name: damage_grade, Length: 329416, dtype: int64

In [21]:
y_l = np.array(y_res - 1)
X_l = np.array(X_scaled)
X_train, X_test, y_train, y_test = tts(X_l, y_l, test_size=0.25, random_state=15)

In [22]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2]), array([86821, 79593, 80648]))

In [23]:
y_test

array([1, 1, 0, ..., 0, 0, 0])

In [71]:
output_size = np.unique(y_l).shape[0]
input_size = X_train.shape[1]

one = LVQ3(input_size=input_size, output_size=output_size, random_state=21, epoch=50, alpha=0.35, beta=0.2, m=0.2,epsilon=0.5)
# one.fit(X_train, y_train)



In [115]:
X_test

array([[-0.54364325, -0.3166606 , -0.35720699, ..., -0.1560186 ,
         0.23849026, -0.09918202],
       [-0.33571196, -0.0995511 , -0.30143126, ...,  0.08318436,
         1.05023523, -0.48813714],
       [-0.57441034, -0.51045358, -0.46829583, ..., -0.0491111 ,
        -0.16905759,  0.00767509],
       ...,
       [-0.4987933 ,  0.30578837, -0.19176103, ...,  0.37645416,
         0.15394247, -0.03741092],
       [ 0.90864365,  0.94536065, -1.18319375, ...,  0.07898511,
         0.25688824,  0.13654522],
       [ 2.54849081, -0.79702766, -0.66221816, ...,  0.44417254,
        -0.47998474, -0.27743699]])

In [123]:
selected = [0, 1, 2, 3, 4, 13, 21]
current_acc = -np.inf

if False:
    for col in range(13, X_train.shape[1]):
        print("Current Col" + str(col))
        selected.append(col)
        print("Training")
        sfs = LVQ3(input_size=len(selected), output_size=output_size, random_state=21, epoch=20, alpha=0.35, beta=0.25, m=0.2,epsilon=0.45)
        sfs.fit(X_train[:,selected], y_train)
        print("Finished Training")
        cur = sfs.predict(X_test[:,selected])
        acc = accuracy_score(cur, y_test)
        print("Best acc: ", current_acc, " Cur acc: ", acc)
        if acc > current_acc:
            current_acc = acc
        else:
            selected.pop()
        print(selected)
    

Current Col13
Training
Finished Training
Best acc:  -inf  Cur acc:  0.34952764893022803
[0, 1, 2, 3, 4, 13]
Current Col14
Training
Finished Training
Best acc:  0.34952764893022803  Cur acc:  0.34952764893022803
[0, 1, 2, 3, 4, 13]
Current Col15
Training
Finished Training
Best acc:  0.34952764893022803  Cur acc:  0.34952764893022803
[0, 1, 2, 3, 4, 13]
Current Col16
Training
Finished Training
Best acc:  0.34952764893022803  Cur acc:  0.34952764893022803
[0, 1, 2, 3, 4, 13]
Current Col17
Training
Finished Training
Best acc:  0.34952764893022803  Cur acc:  0.34952764893022803
[0, 1, 2, 3, 4, 13]
Current Col18
Training
Finished Training
Best acc:  0.34952764893022803  Cur acc:  0.34951550622920563
[0, 1, 2, 3, 4, 13]
Current Col19
Training
Finished Training
Best acc:  0.34952764893022803  Cur acc:  0.34952764893022803
[0, 1, 2, 3, 4, 13]
Current Col20
Training
Finished Training
Best acc:  0.34952764893022803  Cur acc:  0.34952764893022803
[0, 1, 2, 3, 4, 13]
Current Col21
Training
Finished

In [124]:
sfs = LVQ3(input_size=len(selected), output_size=output_size, random_state=21, epoch=100, alpha=0.35, beta=0.25, m=0.2,epsilon=0.45)
sfs.fit(X_train[:,selected], y_train)


In [125]:
sfs.weight

array([[  5.36160014,  -1.2709017 ,   1.95592032,  -0.69444597,
         -1.26329795,  -5.37317725,   3.76285558],
       [-15.87952405,   4.92680221, -26.46711663,  19.22960644,
        -18.65739087, -17.57120294,  -5.15477141],
       [-20.30505947,   0.78634607,  -7.92572669,   9.13053603,
          6.92687833,   9.46306736, -25.04558991]])

In [126]:
y_train

array([0, 2, 0, ..., 0, 0, 0])

In [127]:
res = sfs.predict(X_test[:,selected])
print(classification_report(res, y_test, zero_division=1))

              precision    recall  f1-score   support

           0       1.00      0.35      0.52     82349
           1       0.00      1.00      0.00         0
           2       0.00      0.40      0.00         5

    accuracy                           0.35     82354
   macro avg       0.33      0.58      0.17     82354
weighted avg       1.00      0.35      0.52     82354
